---
layout: post
title:  "Cycle Generative Adversarial Network"
date:   2023-03-24 10:14:54 +0700
categories: jekyll update
---

# Introduction

The first difference between CycleGAN and other GANs is that it is unpaired image translation. Unpaired means that the input images don't look like each other, the translation part only takes care of transfering the style. So it is a style transfering GAN, but the mechanism is different from StyleGAN. 

<img width="611" alt="Screen Shot 2023-03-24 at 11 41 59" src="https://user-images.githubusercontent.com/7457301/227426252-783ffe3c-51f6-41d2-b13e-f2172b2ae76d.png">

The second difference is in the translating mechanism. It has two discriminators $$ D_X $$ and $$ D_Y $$, with two generators (two generating functions) F and G. There are two inputs: X and Y. G translate X into Y, as usual, and F translate Y to X. F is the inverse function of G. The authors add a new term into the loss function called cycle consistency loss. When you translate x to y and back to x', the cycle consistency loss would make sure that x' is highly similar to x. This is call forward consistency loss: x -> G(x) -> F(G(x)) $$ \approx $$ x. We have the composite functions: $$ F \circ G: X \rightarrow X $$. In the same vein, when you translate from y to x and back to y', they cycle consistency loss would make sure that y' is close to y. This is called backward cycle consistency loss: y -> F(y) -> G(F(y)) $$ \approx $$ y. The composite function is $$ G \circ F: Y \rightarrow Y $$. In natural language, this process is equivalent to: translate a sentence from English to Spainish, the translate that Spanish sentence back to English, we need to make sure that we get back the English sentence. This task is actually style transfer, however, style transferring uses a different technique: they match the Gram matrix statistics of pre trained deep features to give the strokes of the style, then use that stroke to paint the input image. Here the authors want to learn the mapping functions between two image collections, rather than just two images.

The adversarial loss becomes 3 losses:

- For the function G that converts X to Y:

with G: X $$ \rightarrow $$ Y

$$ Loss_{GAN}(G,D_Y,X,Y) = E_y{[logD_Y(y)]} + E_x{[log(1-D_Y(G(x))]} $$

The problem becomes $$ min_G max_{D_Y} Loss_{GAN}(G,D_Y,X,Y) $$

- For the function F that converts Y to X:

with F: Y $$ \rightarrow $$ X

$$ min_F max_{D_X} Loss_{GAN}(F,D_X,Y,X) $$

- The cycle consistency loss encourages forward and backward cycle consistency:

$$ Loss_{cycle}(G,F) = E_x{[ \mid \mid F(G(x)) - x \mid \mid_1 ]} + E_y{[ \mid \mid G(F(y)) - y \mid \mid_1]} $$

The full objective function is:

$$ Loss(G,F,D_X,D_Y) = Loss_{GAN}(G,D_Y,X,Y) + Loss_{GAN}(F,D_X,Y,X)+\alpha Loss_{cycle}(G,F)$$

$$ G^*, F^* = arg min_{G,F} max_{D_X, D_Y} Loss(G,F,D_X,D_Y) $$

The authors use a similar to style transferring architecture for the generator: three convolutions, some residual blocks, two convolutions with stride $$ \frac{1}{2} $$ and one convolution to map features to RGB. They use six blocks for 128x128 images and nine blocks for 256x256 and higher resolution images. 70x70 PatchGANs are used for the discriminator. PatchGAN is a GAN that classify patches of the image to be real or fake, not the entire image. This gives equivalent results, but with less computation. 

During training, though, a least squares loss is replacement for the negative log likelihood. In particular, G is trained to minimize $$ E_x{[(D(G(x))-1)^2]} $$ and D to minimize $$ E_y{[(D(y)-1)^2]} + E_x{[D(G(x))^2]} $$. They set $$ \alpha = 10 $$, use stochastic gradient descent (batch size = 1), and learning rate = 0.0002. The same learning rate is applied for the first 100 epochs and linearly decay to zero over the next 100 epochs.

# Code example

We use the provided dataset of horses and zebras in the tensorflow datasets. It is a mix of real images of horses in zebras in different settings. And the generator use the structure of a ResNet. ResNet is short for Residual Network in which they concatenate input directly to a later layer to provide the extra and original feature maps to later layer. This is done because the information is warped / transformed after many convoluted layers, to send the original structuring as additional information to the later layer, they simply let the input skip several connections and concanate directly to the later layer they wish.


#### Input

![horseandzebras](https://user-images.githubusercontent.com/7457301/227464344-370e51bc-4167-44e0-ac63-38b3eb7a8dbb.png)


In [ ]:
class ReflectionPadding2D(layers.Layer):
    """Implements Reflection Padding as a layer.

    Args:
        padding(tuple): Amount of padding for the
        spatial dimensions.

    Returns:
        A padded tensor with the same type as the input tensor.
    """

    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        super().__init__(**kwargs)

    def call(self, input_tensor, mask=None):
        padding_width, padding_height = self.padding
        padding_tensor = [
            [0, 0],
            [padding_height, padding_height],
            [padding_width, padding_width],
            [0, 0],
        ]
        return tf.pad(input_tensor, padding_tensor, mode="REFLECT")


def residual_block(
    x,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(1, 1),
    padding="valid",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    dim = x.shape[-1]
    input_tensor = x

    x = ReflectionPadding2D()(input_tensor)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = activation(x)

    x = ReflectionPadding2D()(x)
    x = layers.Conv2D(
        dim,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.add([input_tensor, x])
    return x


def downsample(
    x,
    filters,
    activation,
    kernel_initializer=kernel_init,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2D(
        filters,
        kernel_size,
        strides=strides,
        kernel_initializer=kernel_initializer,
        padding=padding,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x


def upsample(
    x,
    filters,
    activation,
    kernel_size=(3, 3),
    strides=(2, 2),
    padding="same",
    kernel_initializer=kernel_init,
    gamma_initializer=gamma_init,
    use_bias=False,
):
    x = layers.Conv2DTranspose(
        filters,
        kernel_size,
        strides=strides,
        padding=padding,
        kernel_initializer=kernel_initializer,
        use_bias=use_bias,
    )(x)
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    if activation:
        x = activation(x)
    return x

def get_resnet_generator(
    filters=64,
    num_downsampling_blocks=2,
    num_residual_blocks=9,
    num_upsample_blocks=2,
    gamma_initializer=gamma_init,
    name=None,
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = ReflectionPadding2D(padding=(3, 3))(img_input)
    x = layers.Conv2D(filters, (7, 7), kernel_initializer=kernel_init, use_bias=False)(
        x
    )
    x = tfa.layers.InstanceNormalization(gamma_initializer=gamma_initializer)(x)
    x = layers.Activation("relu")(x)

    # Downsampling
    for _ in range(num_downsampling_blocks):
        filters *= 2
        x = downsample(x, filters=filters, activation=layers.Activation("relu"))

    # Residual blocks
    for _ in range(num_residual_blocks):
        x = residual_block(x, activation=layers.Activation("relu"))

    # Upsampling
    for _ in range(num_upsample_blocks):
        filters //= 2
        x = upsample(x, filters, activation=layers.Activation("relu"))

    # Final block
    x = ReflectionPadding2D(padding=(3, 3))(x)
    x = layers.Conv2D(3, (7, 7), padding="valid")(x)
    x = layers.Activation("tanh")(x)

    model = keras.models.Model(img_input, x, name=name)
    return model

def get_discriminator(
    filters=64, kernel_initializer=kernel_init, num_downsampling=3, name=None
):
    img_input = layers.Input(shape=input_img_size, name=name + "_img_input")
    x = layers.Conv2D(
        filters,
        (4, 4),
        strides=(2, 2),
        padding="same",
        kernel_initializer=kernel_initializer,
    )(img_input)
    x = layers.LeakyReLU(0.2)(x)

    num_filters = filters
    for num_downsample_block in range(3):
        num_filters *= 2
        if num_downsample_block < 2:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(2, 2),
            )
        else:
            x = downsample(
                x,
                filters=num_filters,
                activation=layers.LeakyReLU(0.2),
                kernel_size=(4, 4),
                strides=(1, 1),
            )

    x = layers.Conv2D(
        1, (4, 4), strides=(1, 1), padding="same", kernel_initializer=kernel_initializer
    )(x)

    model = keras.models.Model(inputs=img_input, outputs=x, name=name)
    return model


# Get the generators
gen_G = get_resnet_generator(name="generator_G")
gen_F = get_resnet_generator(name="generator_F")

# Get the discriminators
disc_X = get_discriminator(name="discriminator_X")
disc_Y = get_discriminator(name="discriminator_Y")


#### Generator F

![generatorF](https://user-images.githubusercontent.com/7457301/227467267-1c6cc545-0798-4017-ae98-065ffbe21dcd.png)

#### Generator G

![generatorG](https://user-images.githubusercontent.com/7457301/227467269-02bf69a5-341d-415f-bd17-af01e1baaf7c.png)

#### Discriminator X

![discriminatorX](https://user-images.githubusercontent.com/7457301/227467254-32ad9115-8d80-4e4a-a99f-4dff1db1c0fe.png)

#### Discriminator Y

![discriminatorY](https://user-images.githubusercontent.com/7457301/227467263-967c35cb-8909-4173-9880-f2367ca1891b.png)

#### Results
![horsetozebra](https://user-images.githubusercontent.com/7457301/227464355-f81417be-bb25-450a-9fa6-c8dd3ef7b402.png)

